In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\Lenovo\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
clusters_init = open('wiki-topcats-categories.txt','rt')
filtered_data = pd.read_csv('filtered-wiki-topcats5.txt', sep=" ", header=None) 
clusters_formed = pd.read_csv('cluster5.csv', header = None)

In [8]:
# used_nodes = pd.read_csv('filtered-wiki-nodes5.txt', sep= " ", header = None)
unique_nodes = set(filtered_data[0].values).union(set(filtered_data[1].values))

In [15]:
len(unique_nodes)

150001

In [11]:
number_clusters = []
cluster_category = []
nodes_indexes = []
with open('wiki-topcats-categories.txt','rt') as f: 
    for line in f:
        name, lines = line.split(';')
        nodes = list(map(int,(lines.split())))
        nodes = list(filter(lambda x: int(x) in unique_nodes, nodes))
        if len(nodes) != 0:
            cluster_category.append(name.split(':')[1])
            number_clusters.append(len(nodes))
            nodes_indexes.append(nodes)

In [18]:
len(unique_nodes.difference(set([i[0] for i in idx_nodes])))

0

In [12]:
ids_categories = {idx:cat for idx,cat in enumerate(cluster_category)}
idx_nodes = []
for cat in ids_categories.keys():
    for node in nodes_indexes[cat]:
        idx_nodes.append((node,cat))
df_filtered = pd.DataFrame(idx_nodes, columns = ['id_node', 'id_category'])

In [13]:
len(idx_nodes)

431617

In [14]:
df_filtered.shape

(431617, 2)

In [29]:
export_csv = df_filtered.to_csv ('id_node_cat.csv', index = None, header=True)

In [ ]:
clusters_node_pers = []
# for cluster in clusters_formed[1].unique():
if True: 
    cluster = 279122
    cluster_percents = []
    set_2 = set(clusters_formed[0][clusters_formed[1] == cluster])
    for cat in ids_categories.keys():
        set_1 = set(df_filtered['id_node'][df_filtered.id_category == cat])
        intersections = set_1.intersection(set_2)
        if len(intersections) != 0 and len(set_2): 
            cluster_percents.append((len(intersections)/len(set_2)))
    clusters_node_pers.append(cluster_percents)   
print(np.array(np.max(clusters_node_pers)))

In [30]:
#statistics for filtered original dataset
df_init = pd.DataFrame(list(zip(cluster_category, number_clusters, nodes_indexes)), columns = ['topic', 'number of nodes per cluster','nodes_idx'])
df_init.describe()

,number of nodes per cluster
count,15593.000000
mean,27.680177
std,170.868488
min,1.000000
25%,5.000000
50%,13.000000
75%,31.000000
max,20315.000000


In [31]:
sort_by_number_of_nodes = df_init.sort_values('number of nodes per cluster', ascending=False)
sort_by_number_of_nodes.head()

,topic,number of nodes per cluster,nodes_idx
962,Living_people,20315,"[256, 872, 1032, 1055, 1100, 1115, 1124, 1289,..."
9576,English-language_films,2592,"[19417, 23865, 41891, 43995, 43997, 52272, 630..."
10653,American_film_actors,1517,"[174, 1124, 44820, 53884, 110163, 110164, 1101..."
9585,American_films,1476,"[19415, 19417, 41891, 43995, 63027, 119822, 13..."
12474,Article_Feedback_Pilot,1471,"[4500, 5345, 5413, 5759, 5763, 5803, 5828, 614..."


In [35]:
with open('filtered-wiki-topcats5.txt','rt') as f: 
    for line in f:
        name, lines = line.split(';')
        nodes = list(map(int,(lines.split())))
        nodes = list(filter(lambda x: int(x) in unique_nodes, nodes))
        if len(nodes) != 0:
            filtered_category.append(name.split(':')[1])
            filtered_clusters.append(len(nodes))
            filtered_indexes.append(nodes)

ValueError: not enough values to unpack (expected 2, got 1)

In [33]:
df_shrinked = pd.DataFrame(list(zip(filte)), columns = ['topic', 'number of nodes per cluster','nodes_idx'])
df_shrinked.describe()

,0,1
0,805973,279122
1,1056524,279122
2,984648,279122
3,1683128,279122
4,851471,1650860
5,386896,279122
6,976434,279122
7,475465,279122
8,513652,279122
9,1233822,279122
